In [ ]:
from IPython.display import display, Image

# Convolutional Neural Network with CIFAR

In this tutorial we will train a Convolutional Neural Network (CNN) on CIFAR data.

## Introduction

A [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN, or ConvNet) is a type of [feed-forward](https://en.wikipedia.org/wiki/Feedforward_neural_network) artificial neural network made up of neurons that have learnable weights and biases, very similar to ordinary multi-layer perceptron (MLP) networks. The CNNs take advantage of the spatial nature of the data. In nature, we perceive different objects by their shapes, size and colors. For example, objects in a natural scene are typically edges, corners/vertices (defined by two of more edges), color patches etc. These primitives are often identified using different detectors (e.g., edge detection, color detector) or combination of detectors interacting to facilitate image interpretation (object classification, region of interest detection, scene description etc.) in real world vision related tasks. These detectors are also known as filters. Convolution is a mathematical operator that takes an image and a filter as input and produces a filtered output (representing say egdges, corners, colors etc in the input image).  Historically, these filters are a set of weights that were often hand crafted or modeled with mathematical functions (e.g., [Gaussian](https://en.wikipedia.org/wiki/Gaussian_filter) / [Laplacian](http://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm) / [Canny](https://en.wikipedia.org/wiki/Canny_edge_detector) filter).  The filter outputs are mapped through non-linear activation functions mimicking human brain cells called [neurons](https://en.wikipedia.org/wiki/Neuron).

Convolutional networks provide a machinery to learn these filters from the data directly instead of explicit mathematical models and have been found to be superior (in real world tasks) compared to historically crafted filters.  With convolutional networks, the focus is on learning the filter weights instead of learning individually fully connected pair-wise (between inputs and outputs) weights. In this way, the number of weights to learn is reduced when compared with the traditional MLP networks from the previous tutorials.  In a convolutional network, one learns several filters ranging from few single digits to few thousands depending on the network complexity.

Many of the CNN primitives have been shown to have a conceptually parallel components in brain's [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex). The group of neurons cells in visual cortex emit responses when stimulated. This region is known as the receptive field (RF). Equivalently, in convolution the input region corresponding to the filter dimensions can be considered as the receptive field. Popular deep CNNs or ConvNets (such as [AlexNet](https://en.wikipedia.org/wiki/AlexNet), [VGG](https://arxiv.org/abs/1409.1556), [Inception](http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf), [ResNet](https://arxiv.org/pdf/1512.03385v1.pdf)) that are used for various [computer vision](https://en.wikipedia.org/wiki/Computer_vision) tasks have many of these architectural primitives (inspired from biology).  

In this notebook, we will introduce the convolution operation and gain familiarity with the different parameters in CNNs.

**Problem**:
We will work on the problem of recognizing images in CIFAR data. The CIFAR data comprises of images that belong to one of 10 classes - ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

**Goal**:
Our goal is to train a classifier that will identify the images in the CIFAR dataset. 

**Approach**:

The 5 stages are: Data reading, Data preprocessing, Creating a model, Learning the model parameters and Evaluating (a.k.a. testing/prediction) the model. 
- Data reading: We will use the tensorflow loader
- Data preprocessing: Covered in part A (suggested extension section). 

In this tutorial, we will experiment with two models with different architectural components.

In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
%matplotlib inline

## Data reading
### Note: We are using the Cifar dataset. But the explanation is on the MNIST data set that you are already familiar with. 

The dataset has 60,000 training images and 10,000 test images with each image being 28 x 28 pixels. Thus the number of features is equal to 784 (= 28 x 28 pixels), 1 per pixel. The variable `num_output_classes` is set to 10 corresponding to the number of digits (0-9) in the dataset.

In neural networks demo, as shown below, we have always flattened the input image into a vector.  With convoultional networks, we do not flatten the image in this way.

![](https://www.cntk.ai/jup/cntk103a_MNIST_input.png)

**Input Dimensions**:  

In convolutional networks for images, the input data is often shaped as a 3D matrix (number of channels, image width, height), which preserves the spatial relationship between the pixels. In the figure above, the MNIST image is a single channel (grayscale) data, so the input dimension is specified as a (1, image width, image height) tuple. 

![](https://www.cntk.ai/jup/cntk103d_rgb.png)

Natural scene color images are often presented as Red-Green-Blue (RGB) color channels. The input dimension of such images are specified as a (image width, image height, 3) tuple. If one has RGB input data as a volumetric scan with volume width, volume height and volume depth representing the 3 axes, the input data format would be specified by a tuple of 4 values (volume width, volume height, volume depth, 3). In this way keras enables specification of input images in arbitrary higher-dimensional space.

In [ ]:
# Define the data dimensions
input_dim_model = (32, 32, 3)    # images are 28 x 28 with 1 channel of color (gray)
input_dim = 32*32*3               # used by readers to treat input data as a vector
num_output_classes = 10

from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print('x_train shape: {}, y_train shape: {}'.format(x_train.shape, y_train.shape))

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 4
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)

**Data Format** For our CIFAR data, each sample contains 2 fields: labels and feature, formatted as:

    |labels 0 0 0 1 0 0 0 0 0 0 |features 0 255 0 123 ... 
                                                  (784 integers each representing a pixel gray level)

The labels are [1-hot](https://en.wikipedia.org/wiki/One-hot) encoded (the label representing the output class of 3  becomes `0001000000` since we have 10 classes for the 10 possible digits), where the first index corresponds to digit `0` and the last one corresponds to digit `9`.

![](https://www.cntk.ai/jup/cntk103a_onehot.png)

## CNN Model Creation

CNN is a feedforward network made up of bunch of layers in such a way that the output of one layer becomes the input to the next layer (similar to MLP). In MLP, all possible pairs of input pixels are connected to the output nodes with each pair having a weight, thus leading to a combinatorial explosion of parameters to be learnt and also increasing the possibility of overfitting ([details](http://cs231n.github.io/neural-networks-1/)). Convolution layers take advantage of the spatial arrangement of the pixels and learn multiple filters that significantly reduce the amount of parameters in the network ([details](http://cs231n.github.io/convolutional-networks/)). The size of the filter is a parameter of the convolution layer.  

In this section, we introduce the basics of convolution operations. We show the illustrations in the context of RGB images (3 channels), eventhough the MNIST data we are using in this tutorial is a grayscale image (single channel).

![](https://www.cntk.ai/jup/cntk103d_rgb.png)

### Convolution Layer

A convolution layer is a set of filters. Each filter is defined by a weight (**W**) matrix, and  bias ($b$).

![](https://www.cntk.ai/jup/cntk103d_filterset_v2.png)

These filters are scanned across the image performing the dot product between the weights and corresponding input value ($x$). The bias value is added to the output of the dot product and the resulting sum is optionally mapped through an activation function. This process is illustrated in the following animation.

In [ ]:
Image(url="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif", width= 300)

Convolution layers incorporate following key features:

   - Instead of being fully-connected to all pairs of input and output nodes , each convolution node is **locally-connected** to a subset of input nodes localized to a smaller input region, also referred to as receptive field (RF). The figure above illustrates a small 3 x 3 regions in the image as the RF region. In the case of an RGB, image there would be three such 3 x 3 regions, one each of the 3 color channels. 
   
   
   - Instead of having a single set of weights (as in a Dense layer), convolutional layers have multiple sets (shown in figure with multiple colors), called **filters**. Each filter detects features within each possible RF in the input image.  The output of the convolution is a set of `n` sub-layers (shown in the animation below) where `n` is the number of filters (refer to the above figure).  
   
     
   - Within a sublayer, instead of each node having its own set of weights, a single set of **shared weights** are used by all nodes in that sublayer. This reduces the number of parameters to be learnt and thus overfitting. This also opens the door for several aspects of deep learning which has enabled very practical solutions to be built:
    -- Handling larger images (say 512 x 512)
    - Trying larger filter sizes (corresponding to a larger RF) say 11 x 11
    - Learning more filters (say 128)
    - Explore deeper architectures (100+ layers)
    - Achieve translation invariance (the ability to recognize a feature independent of where they appear in the image). 

### Strides and Pad parameters

**How are filters positioned?** In general, the filters are arranged in overlapping tiles, from left to right, and top to bottom.  Each convolution layer has a parameter to specify the `filter_shape`, specifying the width and height of the filter in case most natural scene images.  There is a parameter (`strides`) that controls the how far to step to right when moving the filters through multiple RF's in a row, and how far to step down when moving to the next row.  The boolean parameter `pad` controls if the input should be padded around the edges to allow a complete tiling of the RF's near the borders. 

The animation above shows the results with a `filter_shape` = (3, 3), `strides` = (2, 2) and `pad` = False. The two animations below show the results when `pad` is set to True. First, with a stride of 2 and second having a stride of 1.
Note: the shape of the output (the teal layer) is different between the two stride settings. Many a times your decision to pad and the stride values to choose are based on the shape of the output layer needed.

In [ ]:
# Plot images with strides of 2 and 1 with padding turned on
images = [("https://www.cntk.ai/jup/cntk103d_padding_strides.gif" , 'With stride = 2'),
          ("https://www.cntk.ai/jup/cntk103d_same_padding_no_strides.gif", 'With stride = 1')]

for im in images:
    print(im[1])
    display(Image(url=im[0], width=200, height=200))

## Building our CNN models

In this CNN tutorial, we first define two containers. One for the input CIFAR image and the second one being the labels corresponding to the 10 digits. When reading the data, the reader automatically maps the 784 pixels per image to a shape defined by `input_dim_model` tuple (in this example it is set to `(28, 28, 1)`).

The first model we build is a simple convolution only network. Here we have two convolutional layers. Since, our task is to detect the 10 classes in the CIFAR database, the output of the network should be a vector of length 10, 1 element corresponding to each digit. This is achieved by projecting the output of the last convolutional layer using a dense layer with the output being `num_output_classes`. We have seen this before with Logistic Regression and MLP where features were mapped to the number of classes in the final layer. Also, note that since we will be using the `softmax` operation that is combined with the `cross entropy` loss function during training (see a few cells below), the final dense layer has no activation function associated with it.

The following figure illustrates the model we are going to build. Note the parameters in the model below are to be experimented with. These are often called network hyperparameters. Increasing the filter shape leads to an increase in the number of model parameters, increases the compute time and helps the model better fit to the data. However, one runs the risk of [overfitting](https://en.wikipedia.org/wiki/Overfitting). Typically, the number of filters in the deeper layers are more than the number of filters in the layers before them. We have chosen 8, 16 for the first and second layers, respectively. These hyperparameters should be experimented with during model building.

![](https://www.cntk.ai/jup/cntk103d_convonly2.png)

In [ ]:
from keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential
def create_model():
    model = Sequential()
    model.add(Conv2D(8, (5,5), strides=(2,2), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(16, (5,5), strides=(2,2), activation='relu', padding='same',))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return(model)

Let us create an instance of the model and inspect the different components of the model. `z` will be used to represent the output of a network. In this model, we use the `relu` activation function. Note: using the `C.layers.default_options` is an elegant way to write concise models. This is key to minimizing modeling errors, saving precious debugging time.

In [ ]:
# Create the model
z = create_model()

# Print the output shapes / parameters of different components
print("Output Shape of the first convolution layer:", z.layers[0].output_shape)
print("Bias Shape of the last dense layer:", z.layers[-1].weights[1])

Understanding number of model parameters to be estimated is key to deep learning since there is a direct dependency on the amount of data one needs to have. You need more data for a model that has larger number of parameters to prevent overfitting. In other words, with a fixed amount of data, one has to constrain the number of parameters. There is no golden rule between the amount of data one needs for a model. However, there are ways one can boost performance of model training with [data augmentation](https://deeplearningmania.quora.com/The-Power-of-Data-Augmentation-2). 

In [ ]:
# Number of parameters in the network
print('Number of parameters in model: {}'.format(z.count_params()))
# You could also use z.summary()

In [ ]:
z.summary()

**Understanding Parameters**:


Our model has 2 convolution layers each having a weight and bias. This adds up to 4 parameter tensors. Additionally the dense layer has weight and bias tensors. Thus, the 6 parameter tensors.

Let us now count the number of parameters:
- *First convolution layer*: There are 8 filters each of size (5 x 5 X 3) where 1 is the number of channels in the input image. This adds up to 600 values in the weight matrix and 8 bias values.


- *Second convolution layer*: There are 16 filters each of size (5 x 5 x 8) where 8 is the number of channels in the input to the second layer (= output of the first layer). This adds up to 3200 values in the weight matrix and 16 bias values.


- *Last dense layer*: There are 8 x 8 x 16 input values and it produces 10 output values corresponding to the 10 classes in the CIFAR dataset. This corresponds to (8 x 8 x 16) x 10 weight values and 10 bias values. This adds up to 10240 weights and 10 biases.

Adding all these up gives the 14074 parameters in the model.

**Knowledge check**: Does the dense layer shape align with the task (CIFAR Image classification)?

** Suggested Activity **
- Try printing shapes and parameters of different network layers,
- Record the training error you get with `relu` as the activation function,
- Now change to `sigmoid` as the activation function and see if you can improve your training error.

## Data reshaping
Notice that `x_train.shape` would give `(50000, 784)` whereas our model input is of shape (28, 28, 1). 
Hence we transform the data into the desired shape

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_train = x_train.reshape((-1, 32, 32, 3))
print('New Shape: {}'.format(x_train.shape))

In [ ]:
from keras.utils.np_utils import to_categorical
y_train_one_hot = to_categorical(y_train)

## Training

We minimize the cross-entropy between the label and predicted probability by the network.

In [ ]:
z.fit(x_train, y_train_one_hot, validation_split=0.1, epochs=50, batch_size=64)

Note, the average test error is very comparable to our training error indicating that our model has good "out of sample" error a.k.a. [generalization error](https://en.wikipedia.org/wiki/Generalization_error). This implies that our model can very effectively deal with previously unseen observations (during the training process). This is key to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting).

### Run evaluation / prediction
We have so far been dealing with aggregate measures of error. Let us now get the probabilities associated with individual data points. For each observation, the `eval` function returns the probability distribution across all the classes. The classifier is trained to recognize digits, hence has 10 classes. First let us route the network output through a `softmax` function. This maps the aggregated activations across the network to probabilities across the 10 classes.

Let us take a small minibatch sample from the test data.

In [ ]:
x_test = x_test.reshape((-1, 32, 32, 3))

sample_test_data = x_test[:10]; sample_test_labels = y_test[:10]

In [ ]:
predicted_labels = list(z.predict_classes(sample_test_data, verbose=0))
# Find the index with the maximum value for the ground truth
print("Label    :", y_test[:10].reshape(-1))
print("Predicted:", predicted_labels)

Let us visualize some of the results

In [ ]:
# Plot a random image
plt.rcParams['figure.figsize'] = (5,5)
sample_number = 9
plt.imshow(sample_test_data[sample_number], cmap="gray_r")
plt.axis('off')

img_pred = classes[predicted_labels[sample_number]]
print("Image Label: ", img_pred)

## Pooling Layer

Often a times, one needs to control the number of parameters especially when having deep networks. For every layer of the convolution layer output (each layer, corresponds to the output of a filter), one can have a pooling layer. Pooling layers are typically introduced to:
- Reduce the dimensionality of the previous layer (speeding up the network),
- Makes the model more tolerant to changes in object location in the image. For example, even when a digit is shifted to one side of the image instead of being in the middle, the classifer would perform the classification task well.

The calculation on a pooling node is much simpler than a normal feedforward node.  It has no weight, bias, or activation function.  It uses a simple aggregation function (like max or average) to compute its output.  The most commonly used function is "max" - a max pooling node simply outputs the maximum of the input values corresponding to the filter position of the input. The figure below shows the input values in a 4 x 4 region. The max pooling window size is 2 x 2 and starts from the top left corner. The maximum value within the window becomes the output of the region. Every time the model is shifted by the amount specified by the stride parameter (as shown in the figure below) and the maximum pooling operation is repeated. 
![maxppool](https://cntk.ai/jup/201/MaxPooling.png)

Another alternative is average pooling, which emits that average value instead of the maximum value. The two different pooling opearations are summarized in the animation below.

In [ ]:
# Plot images with strides of 2 and 1 with padding turned on
images = [("https://www.cntk.ai/jup/c103d_max_pooling.gif" , 'Max pooling'),
          ("https://www.cntk.ai/jup/c103d_average_pooling.gif", 'Average pooling')]

for im in images:
    print(im[1])
    display(Image(url=im[0], width=300, height=300))

## Typical convolution network

![](http://www.cntk.ai/jup/conv103d_mnist-conv-mp.png)

A typical CNN contains a set of alternating convolution and pooling layers followed by a dense output layer for classification. You will find variants of this structure in many classical deep networks (VGG, AlexNet etc).  This is in contrast to the MLP networks, which consist of dense layers followed by a dense output layer.  

The illustrations are presented in the context of 2-dimensional (2D) images, but the concept and the keras components can operate on any dimensional data. The above schematic shows 2 convolution layer and 2 max-pooling layers. A typical strategy is to increase the number of filters in the deeper layers while reducing the spatial size of each intermediate layers.

## Task: Create a network with MaxPooling

Typical convolutional networks have interlacing convolution and max pool layers. The previous model had only convolution layer. In this section, you will create a model with the following architecture.

![](https://www.cntk.ai/jup/cntk103d_conv_max2.png)

You will use the keras [MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d) function from `keras.layers` module to achieve this task. You will edit the `create_model` function below and add the MaxPooling operation. 

In [ ]:
# Modify this model
def create_model():
    model = Sequential()
    model.add(Conv2D(8, (5,5), strides=(2,2), activation='relu', padding='same', input_shape=(28, 28, 1)))
    model.add(Conv2D(16, (5,5), strides=(2,2), activation='relu', padding='same',))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return(model)


**Quiz**: How many parameters do we have in the model with MaxPooling and Convolution? Which of the two models produces lower error rate?


**Exploration Suggestion**
- Does the use of LeakyRelu help improve the error rate?
- What percentage of the parameter does the last dense layer contribute w.r.t. the overall number of parameters for (a) purely two convolutional layer and (b) alternating 2 convolutional and maxpooling layers   

In [ ]:
model = create_model()
model.fit(x_train, y_train, validation_split=0.1, epochs=10, batch_size=64)